In [1]:
import pandas as pd

In [2]:
df_chart_tbl = pd.read_csv('D:\Papers\Cyprus_Paper\Data\o15_empty_chart_max_tbl.csv', index_col=False)
df_max_chart = pd.read_csv('D:\Papers\Cyprus_Paper\Data\o10_chartevent_24_hour_max.csv', index_col=False)
df_data_chart = pd.read_csv('D:\MSc Aegean\Thesis\mimic-iv-1.0\icu\d_items.csv.gz')

In [3]:
"""
Δημιουργώ εκ νέου την στήλη concat
χωρίς κενά ανάμεσα στα στοιχεία.
Θα χρησιμοποιήσω αυτή τη στήλη σαν
index γιατί τα κενά μπορεί να
δημιουργήσουν πρόβλημα.
"""
df_chart_tbl["concat"] = (df_chart_tbl["subject_id"].astype(str)
                          + df_chart_tbl["hadm_id"].astype(str)
                          + df_chart_tbl["charttime"].astype(str))

"""
Αφαιρώ τους χαρακτήρες - και :
από της τιμές της concat για να
αποφύγω πιθανά προβλήματα
"""
df_chart_tbl['concat'] = df_chart_tbl['concat'].str.replace('-','')
df_chart_tbl['concat'] = df_chart_tbl['concat'].str.replace(':','')

In [4]:
# Ορίζω την στήλη concat ως index των γραμμών
df_chart_tbl = df_chart_tbl.set_index('concat')

In [5]:
"""
Δημιουργώ την στήλη concat και στον πίνακα
που περιλαμβάνει τις μετρήσεις για να είναι
το κοινό σημείο ανάμεσα στους δυο πίνακες
"""
df_max_chart["concat"] = (df_max_chart["subject_id"].astype(str)
                           + df_max_chart["hadm_id"].astype(str)
                           + df_max_chart["charttime"].astype(str))


"""
Αφαιρώ τους χαρακτήρες - και :
από της τιμές της concat για να
αποφύγω πιθανά προβλήματα
"""
df_max_chart['concat'] = df_max_chart['concat'].str.replace('-','')
df_max_chart['concat'] = df_max_chart['concat'].str.replace(':','')

In [13]:
df_max_chart

,subject_id,hadm_id,itemid,charttime,Max_Chart,concat
0,10004733,27411876,220045,2174-12-04,107.0,100047332741187621741204
1,10004733,27411876,220045,2174-12-05,117.0,100047332741187621741205
2,10004733,27411876,220045,2174-12-06,111.0,100047332741187621741206
3,10004733,27411876,220045,2174-12-07,91.0,100047332741187621741207
4,10004733,27411876,220045,2174-12-08,140.0,100047332741187621741208
...,...,...,...,...,...,...
2185164,19999987,23865745,228410,2145-11-04,5.0,199999872386574521451104
2185165,19999987,23865745,228411,2145-11-04,5.0,199999872386574521451104
2185166,19999987,23865745,228412,2145-11-04,5.0,199999872386574521451104
2185167,19999987,23865745,228648,2145-11-03,0.0,199999872386574521451103


In [6]:
# Γεμίζω τον πίνακα
for row in df_max_chart.itertuples():
    con = row.concat
    item = row.itemid
    value = row.Max_Chart
    df_chart_tbl.at['{}'.format(con), '{}'.format(item)] = value

In [7]:
# Αποθήκευση του τελικού αρχείου με επικεφαλίδες τους κωδικούς των παρατηρήσεων
df_chart_tbl.to_csv('D:\Papers\Cyprus_Paper\Data\o21_fill_chart_max_itemid_header_tbl.csv', index=False)

In [8]:
df_data_chart = df_data_chart.set_index('itemid')

In [9]:
# Ενοποίηση τιμών σε κελιά
df_data_chart["header"] = ("Max - "
                           + df_data_chart["label"].astype(str)
                           + " ("
                           + df_data_chart["unitname"].astype(str)
                           + ")")

In [10]:
"""
Ελέγχω τις κεφαλίδες και αν δεν ανήκουν στις
τέσσερις πρώτες τιμές κάνω τις αντικατάσταση.
"""
for col in df_chart_tbl.columns:
    if ((col != 'subject_id') and
        (col != 'hadm_id') and
        (col != 'charttime')):
        temp = (df_data_chart.loc[int(col), 'header'])
        df_chart_tbl.rename(columns={'{}'.format(col): '{}'.format(temp)}, inplace=True)

In [11]:
df_chart_tbl

,subject_id,hadm_id,charttime,Max - Heart Rate (bpm),Max - Respiratory Rate (insp/min),Max - O2 saturation pulseoxymetry (%),Max - Non Invasive Blood Pressure systolic (mmHg),Max - Non Invasive Blood Pressure diastolic (mmHg),Max - Non Invasive Blood Pressure mean (mmHg),Max - Arterial Blood Pressure mean (mmHg),...,Max - Resistance (cmH2O/L/seconds),Max - ABI Ankle BP L (mmHg),Max - ABI Brachial BP R (mmHg),Max - ITBVI (PiCCO) (mL/m2),Max - Patient/Family Informed_V1 (nan),Max - Pheresis Catheter Line Tip Cultured (nan),Max - Triple Introducer Line Tip Cultured (nan),Max - Depth at which wedge obtained (PA Line) (cm),Max - Clot Size (cm) (cm),Max - Resting Energy Expenditure (kcal/day)
concat,,,,,,,,,,,,,,,,,,,,,
100047332741187621741204,10004733,27411876,2174-12-04,107.0,22.0,100.0,181.0,112.0,128.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741205,10004733,27411876,2174-12-05,117.0,31.0,98.0,191.0,89.0,110.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741206,10004733,27411876,2174-12-06,111.0,23.0,100.0,174.0,104.0,121.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741207,10004733,27411876,2174-12-07,91.0,23.0,100.0,180.0,129.0,138.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741208,10004733,27411876,2174-12-08,140.0,22.0,99.0,215.0,132.0,147.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994422678531721481125,19999442,26785317,2148-11-25,63.0,18.0,99.0,138.0,88.0,93.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199994422678531721481126,19999442,26785317,2148-11-26,64.0,17.0,100.0,152.0,97.0,108.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199999872386574521451102,19999987,23865745,2145-11-02,96.0,21.0,100.0,115.0,70.0,81.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Αποθήκευση του τελικού αρχείου με επικεφαλίδες τις ονομασίες των παρατηρήσεων
df_chart_tbl.to_csv('D:\Papers\Cyprus_Paper\Data\o22_fill_max_chart_label_header_tbl.csv', index=False)